<p>I will need to scrape the list of neighborhoods and populations off of this wikipedia page 
https://en.wikipedia.org/wiki/List_of_communities_in_Miami-Dade_County,_Florida
</p>

<p>I will then follow the links to each neighborhood and scrape the population density off of those pages for each negighborhoods</p>
    

<p>Import all Necessary libraries</p>

In [498]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import folium

from sklearn import preprocessing
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


#Pandas is not letting me view all rows, changing the 'display.max_rows' setting's value to 'None'
pd.set_option('display.max_rows', None)

In [355]:
#URL of info source
wiki_url = 'https://en.wikipedia.org/wiki/List_of_communities_in_Miami-Dade_County,_Florida'

#create http request to query URL content
wiki_response = requests.get(wiki_url)

#create BeautifulSoup object to parse response content
wikipage = BeautifulSoup(wiki_response.text)

#use pd.read_html to create a list of dataframes
#I used prettify() after realizing beautifulsoup returns the object as text by default, and read_html parses HTML
#_links variables will provide links to page with population density data
miami_dade_muni = pd.read_html(wikipage.find_all("table")[0].prettify())
miami_dade_cdp = pd.read_html(wikipage.find_all("table")[1].prettify())
muni_links = wikipage.find_all("table")[0].find_all("a")
cdp_links = wikipage.find_all("table")[1].find_all("a")

In [356]:
#format miami_dade_muni df
miami_dade_muni = miami_dade_muni[0].drop(labels=["#","Designation","Date incorporated"],axis=1)
miami_dade_muni.rename(columns={"Incorporated Community":"Neighborhood","2010 Population":"Population"}, inplace=True)
#format miami_dade_cdp df
miami_dade_cdp = miami_dade_cdp[0].drop(labels=["#"],axis=1)
miami_dade_cdp.rename(columns={"Census-designated place":"Neighborhood","2010 Population":"Population"}, inplace=True)


In [357]:
miami_dade_neighborhoods_raw = miami_dade_cdp.append(miami_dade_muni)

In [358]:
miami_dade_neighborhoods_raw

,Neighborhood,Population
0,Brownsville,15313
1,Coral Terrace,24376
2,Country Club,47105
3,Country Walk,15997
4,Fisher Island,132
5,Fontainebleau,59764
6,Gladeview,11535
7,Glenvar Heights,16898
8,Golden Glades,33145
9,Goulds,10103


In [359]:
#go through each href in links DF, add density to new dataframe
hood_densities = pd.DataFrame()
wiki_string = 'https://en.wikipedia.org'

for link in muni_links:
    #print(wiki_string+str(link['href']))
    url_link = wiki_string+str(link['href'])
    density_page = BeautifulSoup(requests.get(url_link).text)
    den_amt = density_page.find_all(attrs={"class": "infobox geography vcard"})[0].find_all('th', string=re.compile("Density$"))[0].next_sibling
    hood_densities = hood_densities.append({"Link":wiki_string+link["href"],"Neighborhood":link["title"].split(',')[0],"Density":den_amt}, ignore_index=True)
hood_densities

#Follow each link and scrape the pages of the density data. Store in df hood_densities
for link in cdp_links:
    print(link)
    url_link = wiki_string+str(link['href'])
    density_page = BeautifulSoup(requests.get(url_link).text)
    try:
        den_amt = density_page.find_all(attrs={"class": "infobox geography vcard"})[0].find_all('th', string=re.compile("Density$"))[0].next_sibling
    except:
        den_amt = 'na'
    hood_densities = hood_densities.append({"Link":wiki_string+link["href"],"Neighborhood":link["title"].split(',')[0],"Density":den_amt}, ignore_index=True)


<a href="/wiki/Brownsville,_Florida" title="Brownsville, Florida">Brownsville</a>
<a href="/wiki/Coral_Terrace,_Florida" title="Coral Terrace, Florida">Coral Terrace</a>
<a href="/wiki/Country_Club,_Florida" title="Country Club, Florida">Country Club</a>
<a href="/wiki/Country_Walk,_Florida" title="Country Walk, Florida">Country Walk</a>
<a href="/wiki/Fisher_Island,_Florida" title="Fisher Island, Florida">Fisher Island</a>
<a href="/wiki/Fontainebleau,_Florida" title="Fontainebleau, Florida">Fontainebleau</a>
<a href="/wiki/Gladeview,_Florida" title="Gladeview, Florida">Gladeview</a>
<a href="/wiki/Glenvar_Heights,_Florida" title="Glenvar Heights, Florida">Glenvar Heights</a>
<a href="/wiki/Golden_Glades,_Florida" title="Golden Glades, Florida">Golden Glades</a>
<a href="/wiki/Goulds,_Florida" title="Goulds, Florida">Goulds</a>
<a href="/wiki/Homestead_Base,_Florida" title="Homestead Base, Florida">Homestead Base</a>
<a href="/wiki/Ives_Estates,_Florida" title="Ives Estates, Florida">

In [347]:
hood_densities

,Density,Link,Neighborhood
0,"[13,983.74/sq mi (5,398.59/km, [2], )]","https://en.wikipedia.org/wiki/Aventura,_Florida",Aventura
1,"[7,731.07/sq mi (2,982.07/km, [2], )]","https://en.wikipedia.org/wiki/Bal_Harbour,_Flo...",Bal Harbour
2,"[14,628.79/sq mi (5,642.92/km, [2], )]",https://en.wikipedia.org/wiki/Bay_Harbor_Islan...,Bay Harbor Islands
3,"[4,943.55/sq mi (1,909.08/km, [2], )]","https://en.wikipedia.org/wiki/Biscayne_Park,_F...",Biscayne Park
4,"[3,844.67/sq mi (1,484.43/km, [2], )]","https://en.wikipedia.org/wiki/Coral_Gables,_Fl...",Coral Gables
5,"[4,421.77/sq mi (1,707.32/km, [2], )]","https://en.wikipedia.org/wiki/Cutler_Bay,_Florida",Cutler Bay
6,"[4,757.63/sq mi (1,836.99/km, [2], )]","https://en.wikipedia.org/wiki/Doral,_Florida",Doral
7,"[5,778.31/sq mi (2,230.28/km, [2], )]","https://en.wikipedia.org/wiki/El_Portal,_Florida",El Portal
8,"[1,965.11/sq mi (758.79/km, [2], )]","https://en.wikipedia.org/wiki/Florida_City,_Fl...",Florida City
9,"[2,861.96/sq mi (1,106.43/km, [2], )]","https://en.wikipedia.org/wiki/Golden_Beach,_Fl...",Golden Beach


In [360]:
#Drop Fisher Island. Don't have a boat, and no density data
hood_densities.drop([38], axis=0, inplace=True)
hood_densities.loc[hood_densities['Neighborhood']=="Fontainebleau","Neighborhood"] = "Fountainbleau"
hood_densities

,Density,Link,Neighborhood
0,"[13,983.74/sq mi (5,398.59/km, [2], )]","https://en.wikipedia.org/wiki/Aventura,_Florida",Aventura
1,"[7,731.07/sq mi (2,982.07/km, [2], )]","https://en.wikipedia.org/wiki/Bal_Harbour,_Flo...",Bal Harbour
2,"[14,628.79/sq mi (5,642.92/km, [2], )]",https://en.wikipedia.org/wiki/Bay_Harbor_Islan...,Bay Harbor Islands
3,"[4,943.55/sq mi (1,909.08/km, [2], )]","https://en.wikipedia.org/wiki/Biscayne_Park,_F...",Biscayne Park
4,"[3,844.67/sq mi (1,484.43/km, [2], )]","https://en.wikipedia.org/wiki/Coral_Gables,_Fl...",Coral Gables
5,"[4,421.77/sq mi (1,707.32/km, [2], )]","https://en.wikipedia.org/wiki/Cutler_Bay,_Florida",Cutler Bay
6,"[4,757.63/sq mi (1,836.99/km, [2], )]","https://en.wikipedia.org/wiki/Doral,_Florida",Doral
7,"[5,778.31/sq mi (2,230.28/km, [2], )]","https://en.wikipedia.org/wiki/El_Portal,_Florida",El Portal
8,"[1,965.11/sq mi (758.79/km, [2], )]","https://en.wikipedia.org/wiki/Florida_City,_Fl...",Florida City
9,"[2,861.96/sq mi (1,106.43/km, [2], )]","https://en.wikipedia.org/wiki/Golden_Beach,_Fl...",Golden Beach


In [361]:
#clean up the density values, add in population, 
extracted_densities = []

#extract the per mile density value and save it to a list as a float
for dens in hood_densities['Density']:
    try: 
        den_str = dens.text.split('/')[0].replace(",","")
    except: 
        den_str = None
    extracted_densities.append(float(den_str))

#Append the list as a new column to our df
hood_densities['Density_num'] = extracted_densities

#Merge df (hood_densities) with miami_dade_neighborhoods
miami_dade_neighborhoods = hood_densities.merge(miami_dade_neighborhoods_raw.set_index('Neighborhood'), on = 'Neighborhood').drop(["Link","Density"], axis=1)

miami_dade_neighborhoods

,Neighborhood,Density_num,Population
0,Aventura,13983.74,35762
1,Bal Harbour,7731.07,2513
2,Bay Harbor Islands,14628.79,5628
3,Biscayne Park,4943.55,3055
4,Coral Gables,3844.67,46780
5,Cutler Bay,4421.77,40286
6,Doral,4757.63,45704
7,El Portal,5778.31,2325
8,Florida City,1965.11,11245
9,Golden Beach,2861.96,919


In [362]:
#Foursquare Creds
CLIENT_ID = 'HIDDEN' # your Foursquare ID
CLIENT_SECRET = 'HIDDEN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50

In [363]:
def getNearbyVenues(names, near):
    
    venues_list=[]
    for name, loc in zip(names, near):
        print(name)
        loc_key = loc.replace(" ","%20")+"%2C%20FL"
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            loc_key, 
            LIMIT)
            
        # make the GET request
        print(url) #- had to print url to visit directly for troubleshooting. i learned i had interpretted latitude and longitude results from geocoder in the worng order 
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood','Venue','Venue Latitude','Venue Longitude','Venue Category']
    
    return(nearby_venues)

In [364]:
miami_venues = getNearbyVenues(names=miami_dade_neighborhoods['Neighborhood'],
                               near=miami_dade_neighborhoods['Neighborhood'])
miami_venues.head()

Aventura
https://api.foursquare.com/v2/venues/explore?&client_id=C13Q5GJLF3P3NCV2PWSK1LLHPOVJZ5QTHDAQPQTMOHYLUGBJ&client_secret=MHBAEXFXLIPRD35KHGXIF5PXIDMMX3HH0WVZUIBIZASGOYBN&v=20180604&near=Aventura%2C%20FL&limit=50
Bal Harbour
https://api.foursquare.com/v2/venues/explore?&client_id=C13Q5GJLF3P3NCV2PWSK1LLHPOVJZ5QTHDAQPQTMOHYLUGBJ&client_secret=MHBAEXFXLIPRD35KHGXIF5PXIDMMX3HH0WVZUIBIZASGOYBN&v=20180604&near=Bal%20Harbour%2C%20FL&limit=50
Bay Harbor Islands
https://api.foursquare.com/v2/venues/explore?&client_id=C13Q5GJLF3P3NCV2PWSK1LLHPOVJZ5QTHDAQPQTMOHYLUGBJ&client_secret=MHBAEXFXLIPRD35KHGXIF5PXIDMMX3HH0WVZUIBIZASGOYBN&v=20180604&near=Bay%20Harbor%20Islands%2C%20FL&limit=50
Biscayne Park
https://api.foursquare.com/v2/venues/explore?&client_id=C13Q5GJLF3P3NCV2PWSK1LLHPOVJZ5QTHDAQPQTMOHYLUGBJ&client_secret=MHBAEXFXLIPRD35KHGXIF5PXIDMMX3HH0WVZUIBIZASGOYBN&v=20180604&near=Biscayne%20Park%2C%20FL&limit=50
Coral Gables
https://api.foursquare.com/v2/venues/explore?&client_id=C13Q5GJLF3P

,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aventura,Houston's,25.936259,-80.148407,American Restaurant
1,Aventura,Nordstrom,25.957253,-80.145240,Women's Store
2,Aventura,Mystic Pointe Tower,25.953201,-80.127895,Residential Building (Apartment / Condo)
3,Aventura,Aventura Mall,25.956054,-80.146718,Shopping Mall
4,Aventura,Whole Foods Market,25.971296,-80.141222,Grocery Store


In [372]:
miami_venues.to_csv("miami_venues")

In [369]:
miami_venues.groupby('Neighborhood').count()

,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,
Aventura,50,50,50,50
Bal Harbour,50,50,50,50
Bay Harbor Islands,22,22,22,22
Biscayne Park,12,12,12,12
Brownsville,14,14,14,14
Coral Gables,50,50,50,50
Coral Terrace,50,50,50,50
Country Club,50,50,50,50
Country Walk,42,42,42,42


In [378]:
# break the Venue Category out into dummy columns 
miami_onehot = pd.get_dummies(miami_venues['Venue Category'], prefix="", prefix_sep="")

# add Neighborhood column to dataframe
miami_onehot.drop("Neighborhood", axis=1, inplace=True)
miami_onehot.insert(0,'Neighborhood',miami_venues['Neighborhood'])

miami_grouped = miami_onehot.groupby('Neighborhood').sum().reset_index()
miami_grouped

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Water Park,Weight Loss Center,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Aventura,0,0,0,0,2,0,0,0,1,...,0,0,0,1,0,0,1,0,0,0
1,Bal Harbour,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,2,0,0,0
2,Bay Harbor Islands,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Biscayne Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Brownsville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Coral Gables,0,0,0,0,3,0,0,0,1,...,0,0,0,0,0,0,2,0,0,0
6,Coral Terrace,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Country Club,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,Country Walk,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Cutler Bay,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0


In [419]:
#We only need the ratios of restuarants:people, and pizzaplaces:restuarants
#restuarants appear with 'restuarant' and 'place' in name
#pizza places appear as pizza place

ratios = pd.DataFrame(columns=["Neighborhood", "Restuarants to People", "Pizza Places to Restuarants"])

#isolate all columns with restuarant or place in name
restuarant_columns=[]
for col in miami_grouped.columns:
    if "Restaurant" in col or "Place" in col or "Steakhouse" in col or "Neighborhood" in col:
        restuarant_columns.append(col)

restuarant_columns

['Neighborhood',
 'African Restaurant',
 'American Restaurant',
 'Arepa Restaurant',
 'Argentinian Restaurant',
 'Asian Restaurant',
 'Brazilian Restaurant',
 'Burrito Place',
 'Cajun / Creole Restaurant',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Cuban Restaurant',
 'Eastern European Restaurant',
 'Falafel Restaurant',
 'Fast Food Restaurant',
 'Fondue Restaurant',
 'French Restaurant',
 'Greek Restaurant',
 'Indian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Kosher Restaurant',
 'Latin American Restaurant',
 'Lebanese Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'New American Restaurant',
 'Paella Restaurant',
 'Persian Restaurant',
 'Peruvian Restaurant',
 'Pizza Place',
 'Portuguese Restaurant',
 'Restaurant',
 'Russian Restaurant',
 'Salad Place',
 'Sandwich Place',
 'Seafood Restaurant',
 'Snack Place',
 'South American Restaurant',
 'Southern / Soul Food Restaurant',
 'Spanish R

In [429]:
#Creat ratios df
for hood in miami_dade_neighborhoods.itertuples():
    print(hood)
    rest_to_ppl = miami_grouped[restuarant_columns].set_index('Neighborhood').loc[hood.Neighborhood].sum()/hood.Density_num
    print(rest_to_ppl," - sum: ",miami_grouped[restuarant_columns].set_index('Neighborhood').loc[hood.Neighborhood].sum(),", density: ",hood.Density_num)
    pizza_to_rest = miami_grouped[["Neighborhood", "Pizza Place"]].set_index('Neighborhood').loc[hood.Neighborhood].sum()/miami_grouped[restuarant_columns].set_index('Neighborhood').loc[hood.Neighborhood].sum()
    print(pizza_to_rest, " - pizza: ", miami_grouped[["Neighborhood", "Pizza Place"]].set_index('Neighborhood').loc[hood.Neighborhood].sum(), ", rest: ",miami_grouped[restuarant_columns].set_index('Neighborhood').loc[hood.Neighborhood].sum()  )
    ratios = ratios.append({"Neighborhood":hood.Neighborhood,"Restuarants to People":rest_to_ppl,"Pizza Places to Restuarants":pizza_to_rest},ignore_index=True)
    
    

Pandas(Index=0, Neighborhood='Aventura', Density_num=13983.74, Population=35762)
0.0010011627790562467  - sum:  14 , density:  13983.74
0.07142857142857142  - pizza:  1 , rest:  14
Pandas(Index=1, Neighborhood='Bal Harbour', Density_num=7731.07, Population=2513)
0.0010347856118234605  - sum:  8 , density:  7731.07
0.0  - pizza:  0 , rest:  8
Pandas(Index=2, Neighborhood='Bay Harbor Islands', Density_num=14628.79, Population=5628)
0.0006835835363006783  - sum:  10 , density:  14628.79
0.1  - pizza:  1 , rest:  10
Pandas(Index=3, Neighborhood='Biscayne Park', Density_num=4943.55, Population=3055)
0.000809135135681848  - sum:  4 , density:  4943.55
0.0  - pizza:  0 , rest:  4
Pandas(Index=4, Neighborhood='Coral Gables', Density_num=3844.67, Population=46780)
0.005462107280989005  - sum:  21 , density:  3844.67
0.09523809523809523  - pizza:  2 , rest:  21
Pandas(Index=5, Neighborhood='Cutler Bay', Density_num=4421.77, Population=40286)
0.002261537800473566  - sum:  10 , density:  4421.77
0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """


Pandas(Index=14, Neighborhood='Key Biscayne', Density_num=10301.52, Population=12344)
0.0014560957994548377  - sum:  15 , density:  10301.52
0.0  - pizza:  0 , rest:  15
Pandas(Index=15, Neighborhood='Medley', Density_num=175.06, Population=838)
0.07426025362732777  - sum:  13 , density:  175.06
0.07692307692307693  - pizza:  1 , rest:  13
Pandas(Index=16, Neighborhood='Miami', Density_num=13000.42, Population=399457)
0.0010768882851477106  - sum:  14 , density:  13000.42
0.0  - pizza:  0 , rest:  14
Pandas(Index=17, Neighborhood='Miami Beach', Density_num=11554.01, Population=87779)
0.0008655003760599134  - sum:  10 , density:  11554.01
0.0  - pizza:  0 , rest:  10
Pandas(Index=18, Neighborhood='Miami Gardens', Density_num=6034.4, Population=107167)
0.0028171814927747583  - sum:  17 , density:  6034.4
0.058823529411764705  - pizza:  1 , rest:  17
Pandas(Index=19, Neighborhood='Miami Lakes', Density_num=5544.81, Population=29361)
0.0034266277834587655  - sum:  19 , density:  5544.81
0.

In [434]:
#merge ratios and miami_dade_neighborhoods
miami_dade_neighborhoods_final = miami_dade_neighborhoods.merge(ratios.set_index('Neighborhood'), on = 'Neighborhood')

miami_dade_neighborhoods_final


,Neighborhood,Density_num,Population,Restuarants to People,Pizza Places to Restuarants
0,Aventura,13983.74,35762,0.001001,0.071429
1,Bal Harbour,7731.07,2513,0.001035,0.000000
2,Bay Harbor Islands,14628.79,5628,0.000684,0.100000
3,Biscayne Park,4943.55,3055,0.000809,0.000000
4,Coral Gables,3844.67,46780,0.005462,0.095238
5,Cutler Bay,4421.77,40286,0.002262,0.000000
6,Doral,4757.63,45704,0.004624,0.045455
7,El Portal,5778.31,2325,0.000346,0.000000
8,Florida City,1965.11,11245,0.005598,0.090909
9,Golden Beach,2861.96,919,0.000000,NaN


In [495]:
#Turn NaN to 0
miami_dade_neighborhoods_final['Pizza Places to Restuarants'] = miami_dade_neighborhoods_final['Pizza Places to Restuarants'].fillna(0)

#Remove neighborhoods with less than 10000 people
miami_dade_neighborhoods_final = miami_dade_neighborhoods_final[miami_dade_neighborhoods_final['Population']>5000]

miami_dade_neighborhoods_final

,Neighborhood,Density_num,Population,Restuarants to People,Pizza Places to Restuarants
0,Aventura,13983.74,35762,0.001001,0.071429
2,Bay Harbor Islands,14628.79,5628,0.000684,0.100000
4,Coral Gables,3844.67,46780,0.005462,0.095238
5,Cutler Bay,4421.77,40286,0.002262,0.000000
6,Doral,4757.63,45704,0.004624,0.045455
8,Florida City,1965.11,11245,0.005598,0.090909
10,Hialeah,10812.24,224669,0.001757,0.052632
11,Hialeah Gardens,7285.54,21744,0.002745,0.050000
12,Homestead,4609.06,60512,0.004122,0.157895
14,Key Biscayne,10301.52,12344,0.001456,0.000000


In [496]:
#Normalizing values in the dataframe. Using minMax scaler to keep values in range 0,1
normalized_df = pd.DataFrame()
scaler = preprocessing.MinMaxScaler()

for col in miami_dade_neighborhoods_final.columns:
    scaler = preprocessing.MinMaxScaler()
    if col in ['Neighborhood']:
        normalized_df[col] = miami_dade_neighborhoods_final[col]
    else:
        normalized_df[col] = scaler.fit_transform(miami_dade_neighborhoods_final[[col]])
    print(type(col))
normalized_df


<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


,Neighborhood,Density_num,Population,Restuarants to People,Pizza Places to Restuarants
0,Aventura,0.617920,0.077394,0.081149,0.142857
2,Bay Harbor Islands,0.649550,0.000951,0.050038,0.200000
4,Coral Gables,0.120748,0.105344,0.518153,0.190476
5,Cutler Bay,0.149046,0.088870,0.204618,0.000000
6,Doral,0.165515,0.102614,0.436065,0.090909
8,Florida City,0.028583,0.015200,0.531431,0.181818
10,Hialeah,0.462405,0.556605,0.155218,0.105263
11,Hialeah Gardens,0.289472,0.041834,0.251995,0.100000
12,Homestead,0.158230,0.140179,0.386904,0.315789
14,Key Biscayne,0.437361,0.017988,0.125715,0.000000


In [499]:
#These are the weights given to the criteria. 
#40% of the decision will be based on the area's population density, 
#10% on the area's overall population
#30% on the area's restuarant to people ratio, 
#20% on the area's pizza place to restuarant ratio, a lower ratio being more favorable.
weights = [4,1,3,-2]
print(weights)


#Multiply the neighborhoods by the weights and then take the weighted average
weighted_df = normalized_df.drop(labels='Neighborhood', axis=1)*weights
weighted_df.sum(axis=1)
weighted_df['Neighborhood'] = miami_dade_neighborhoods_final['Neighborhood']
weighted_df['scores']=weighted_df.sum(axis=1)
weighted_df.sort_values(by='scores', ascending=False)

[4, 1, 3, -2]


,Density_num,Population,Restuarants to People,Pizza Places to Restuarants,Neighborhood,scores
29,3.959024,0.039520,0.139993,-0.000000,Sunny Isles Beach,4.138538
16,2.278811,1.000000,0.265701,-0.000000,Miami,3.544512
22,4.000000,0.004779,0.124667,-0.615385,North Bay Village,3.514062
62,0.087891,0.024845,3.000000,-0.210526,Three Lakes,2.902210
10,1.849619,0.556605,0.465655,-0.210526,Hialeah,2.661353
0,2.471680,0.077394,0.243446,-0.285714,Aventura,2.506806
17,1.995110,0.209348,0.203577,-0.000000,Miami Beach,2.408035
2,2.598201,0.000951,0.150114,-0.400000,Bay Harbor Islands,2.349266
14,1.749445,0.017988,0.377145,-0.000000,Key Biscayne,2.144579
58,0.663290,0.028249,1.676548,-0.285714,Sunset,2.082373
